<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Imports</h1>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Constants and Global Variables</h1>

In [ ]:
audioAndText = pd.read_csv('Collected Datasets/audioAndText.csv')
text = pd.read_csv('Collected Datasets/text.csv')

<h1 style="text-align:center; font-family:Georgia; font-weight:bold; ">Cleaning and Preprocessing</h1>